In [1]:
from __future__ import division

In [2]:
import os
import random
import math
import datetime
import time

In [3]:
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


# Train a new model
- Load the features/targets
- Create a model
- Save it with a nice name to 'models/{}'

In [ ]:
features = pd.read_csv('training_data/random-2017-10-21-12:57:12/features.csv',
                       names=['game_idx'] + range(42),
                       dtype='int')

targets = pd.read_csv('training_data/random-2017-10-21-12:57:12/targets.csv',
                      names=['game_idx', 'win', 'lose', 'draw'],
                      dtype='int')

In [31]:
# Split into training and holdout,
# ensuring separate games 

train_games = pd.Series(features.game_idx.unique()).sample(frac=0.8).values #games = #features.game_idx.unique()

features_train = features[features.game_idx.isin(train_games)].drop('game_idx', axis=1)
targets_train = targets[targets.game_idx.isin(train_games)].drop('game_idx', axis=1)

features_test = features[~features.game_idx.isin(train_games)].drop('game_idx', axis=1)
targets_test = targets[~targets.game_idx.isin(train_games)].drop('game_idx', axis=1)

In [32]:
print len(features), len(features_train), len(features_test)
print len(targets), len(targets_train), len(targets_test)

414566 331848 82718
414566 331848 82718


In [34]:
def get_batch_iter(batch_size, batch_idx, dfs):
    
    length = len(dfs[0])
    for df in dfs:
        assert len(df) == length
        
    batches_per_df = int(math.ceil(length / batch_size))
    
    local_idx = batch_idx % batches_per_df
    
    start = local_idx*batch_size
    end = (local_idx+1)*batch_size
    
    return [df.iloc[start:end] for df in dfs]


def get_batch_random(batch_size, _, dfs):    
    mask = pd.Series(dfs[0].index).sample(n=batch_size, replace=False)    
    return [df.loc[mask] for df in dfs]


def get_batch(batch_size, batch_idx, dfs, how='iter'):
    if how == 'iter':
        return get_batch_iter(batch_size, batch_idx, dfs)
    elif how == 'random':
        return get_batch_random(batch_size, batch_idx, dfs)
    else:
        raise Exception()

In [42]:
def run(graph, output_prefix, batch_size, num_batches, batch_how='iter'):
    
    with tf.Session(graph=graph) as sess:
    
        K.set_session(sess)
        sess.run(init_op)
    
        #suffix = datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
        #name = output_prefix #'{}_{}'.format(output_prefix, suffix)
        #save_name = "{}/model".format(name) #, suffix)

        train_writer = tf.summary.FileWriter(output_prefix, sess.graph)
        
        print "Running {}".format(output_prefix)
            
        t = time.time()
        delta_t = 0
        
        for i in range(num_batches):
        
            if i % 1000 == 0:
                
                delta_t = time.time() - t
                t = time.time()
                    
                acc = acc_value.eval(feed_dict={board: features_test, outcome: targets_test}).mean()
                print "Batch {} Hold-Out Accuracy: {} Time taken: {:.1f}s".format(i, acc, delta_t)

                summary = sess.run(all_summaries, feed_dict={board: features_test, outcome: targets_test})
                train_writer.add_summary(summary, i)            
            
            batch = get_batch(batch_size, i, [features_train, targets_train], how=batch_how)        
            train_step.run(feed_dict={board: batch[0], outcome: batch[1]})
        
        print "DONE TRAINING"
        print "FINAL ACCURACY: {}".format(acc)
        train_writer.close()
        
        model_dir = '{}/model'.format(output_prefix)
        print "SAVING TO: {}".format(model_dir)
        tf.train.Saver().save(sess, model_dir)

## Attempt 1: Normal dense fully-connected neural network

In [36]:
from keras.layers import Dense
from keras.objectives import categorical_crossentropy
from keras.metrics import categorical_accuracy as accuracy
from keras.regularizers import l2

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():

    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board')
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')

    # Fully connected layers
    x = Dense(512, activation='tanh')(board)    
    x = Dense(512, activation='tanh')(x) 
    x = Dense(512, activation='tanh')(x)
    x = Dense(48, activation='tanh')(x)

    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
        
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)    
    
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [ ]:
run(graph, './models/dense_{}/'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")),
    batch_size=200,
    num_batches=10000)

Running ./models/dense_2017_10_21_13:40:29/
Batch 0 Hold-Out Accuracy: 0.328731358051 Time taken: 0.0s
Batch 1000 Hold-Out Accuracy: 0.546446979046 Time taken: 15.0s
Batch 2000 Hold-Out Accuracy: 0.540692448616 Time taken: 15.8s
Batch 3000 Hold-Out Accuracy: 0.556529402733 Time taken: 16.4s
Batch 4000 Hold-Out Accuracy: 0.555465579033 Time taken: 16.6s


## Attempt 2: A number of parallel CovNets

In [ ]:
from keras.layers import Conv2D, Flatten
from keras.layers.core import Reshape
from keras.layers.merge import Concatenate

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():
    
    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board') 
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')
    
    d1 = Dense(512, activation='tanh')(board) 
        
    # The input data is [col0=[row_0, row_1, ...], col1=[row_0, row_1], ...]
    rs = Reshape((7, 6, 1), input_shape=(42,))(board)    
    
    c1 = Flatten()(Conv2D(1, kernel_size=(4, 4), activation='relu', input_shape=(7, 6, 1))(rs))
    c2 = Flatten()(Conv2D(1, kernel_size=(1, 4), activation='relu', input_shape=(7, 6, 1))(rs))
    c3 = Flatten()(Conv2D(1, kernel_size=(4, 1), activation='relu', input_shape=(7, 6, 1))(rs))
    c4 = Flatten()(Conv2D(1, kernel_size=(5, 5), activation='relu', input_shape=(7, 6, 1))(rs))
    c5 = Flatten()(Conv2D(1, kernel_size=(3, 3), activation='relu', input_shape=(7, 6, 1))(rs))    
    c6 = Flatten()(Conv2D(1, kernel_size=(2, 2), activation='relu', input_shape=(7, 6, 1))(rs))

    merged = Concatenate()([d1, c1, c2, c3, c4, c5, c6])
    
    x = Dense(256, activation='tanh')(merged)         
    x = Dense(128, activation='tanh')(x)    
    x = Dense(12, activation='tanh')(x)
    
    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
    
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)
        
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)    
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [ ]:
sess = run(graph, './train/conv2d', batch_size=250, num_batches=80000, save_prefix='./models/cov2d')